In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from lxml import etree
import pandas as pd
import json

In [2]:
headers = {
            'User-Agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36'
        }

In [3]:
url='https://news.qq.com/zt2020/page/feiyan.htm'
response = requests.get(url,headers = headers)

In [4]:
# 判断是否响应成功
response.status_code

200

In [5]:
# 指定自己Webdirver的路径
driver_url = r'D:\\youdao\\chromedriver.exe'
# 打开浏览器窗口
driver = webdriver.Chrome(executable_path=driver_url)
# 打开网页
driver.get(url)

In [6]:
# 从dirver.page_source属性，获得网页html
html = driver.page_source

In [7]:
# 关闭浏览器窗口
driver.close()

In [8]:
# 定位并读取数据所在的table   
soup = BeautifulSoup(html,'lxml')  # 使用lxml解析器去解析html 
table = soup.find('div',class_="listWraper filterGray").find('table')

In [9]:
type(table)

bs4.element.Tag

In [10]:
# prettify()方法读到得是列表，由于只有一个表格，所以[0]
data_df = pd.read_html(table.prettify())[0]

In [11]:
data_df.head()

,地区,新增确诊,累计确诊,治愈,死亡,疫情
0,美国,56130,4371839,2090129,149849,详情
1,巴西,24578,2419091,1812913,87004,详情
2,印度,49931,1435616,917568,32771,详情
3,俄罗斯,5741,811073,599172,13249,详情
4,南非,11233,445433,265077,6769,详情


In [12]:
# 去除最后一列无关项
data_df = data_df.iloc[:,0:len(data_df.columns)-1]
#data_df.shape[0]
data_df.head()

,地区,新增确诊,累计确诊,治愈,死亡
0,美国,56130,4371839,2090129,149849
1,巴西,24578,2419091,1812913,87004
2,印度,49931,1435616,917568,32771
3,俄罗斯,5741,811073,599172,13249
4,南非,11233,445433,265077,6769


In [13]:
#保存为json格式
#data_df.to_json('./data/data_df.json')
#import json
# 从文件中读入. json.load方法
#with open('./data/data_df.json','r',encoding='utf-8') as json_file:
#    data_dict = json.load(json_file) 

In [14]:
# series 可以直接转为 list
countrys = list(data_df['地区'])
existing_confirm=list(data_df['累计确诊'])

In [15]:
# 读入一个国家名称的中英映射文件，用于中英转化。
import json
with open('name_map.json','r',encoding='utf-8') as json_file:
    # json.dump()和json.load()用于读写。 dumps()和loads()用于dict和json的转换
    name_map = json.load(json_file)

FileNotFoundError: [Errno 2] No such file or directory: 'name_map.json'

In [ ]:
# 键值互换,不用管，暂时注释掉  
# name_map = dict(zip(name_map.values(),name_map.keys()))

In [ ]:
print( list(name_map.keys())[0] )
print( list(name_map.values())[0] )

In [ ]:
# 将国家名称改为英文   用 dict.get()方法，因为不能保证一定有完整的映射关系。
countrys = [name_map.get(name) for name in countrys]

In [ ]:
list_=[list(z) for z in zip(countrys,existing_confirm)]
# 剔除国家名称为空的数据
list_data=[x for x in list_ if x[0]!=None]

In [ ]:
#list_data

In [ ]:
import json
import datetime
from pyecharts.charts import Map
from pyecharts import options as opts


# 疫情地图
# 自定义的每一段的范围，以及每一段的特别的样式。
pieces = [
    {'min': 100000, 'color': '#540d0d'},
    {'max': 99999, 'min': 10000, 'color': '#9c1414'},
    {'max': 9999, 'min': 5000, 'color': '#d92727'},
    {'max': 4999, 'min': 1000,'color': '#ed3232'},
    {'max': 999, 'min': 100, 'color': '#f27777'},
    {'max': 99, 'min': 10, 'color': '#f7adad'},
    {'max': 9, 'min': 1, 'color': '#f7e4e4'},

]


m = Map()
m.add("累计确诊", list_data, 'world')

#系列配置项,可配置图元样式、文字样式、标签样式、点线样式等
m.set_series_opts(label_opts=opts.LabelOpts(is_show=False))
#全局配置项,可配置标题、动画、坐标轴、图例等
m.set_global_opts(title_opts=opts.TitleOpts(title='全球实时确诊数据',
                                            subtitle='数据来源：腾讯网'),
                  legend_opts=opts.LegendOpts(is_show=False),
                  visualmap_opts=opts.VisualMapOpts(pieces=pieces,is_piecewise=True,   
                                                    is_show=True))       
#render（）生成本地 HTML 文件
m.render(path='./data/确诊数据.html')


In [ ]:
import pyecharts.options as opts
from pyecharts.charts import MapGlobe
from pyecharts.globals import ThemeType

c = (
    MapGlobe()
    .add_schema()
    .add(
        maptype="world",
        series_name="World",
        data_pair=list_data,
    )
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(title_opts=opts.TitleOpts(title='全球实时确诊数据',
            subtitle='数据来源：腾讯网'),
            legend_opts=opts.LegendOpts(
                is_show=False),
                visualmap_opts=opts.VisualMapOpts(
                pieces=pieces,
                is_piecewise=True,   
                is_show=True,

            )
    )
    .render("map_globe_base.html")
)